TFM

In [4]:
import pandas as pd
import re

In [5]:
# Cargar archivos
# Ruta al archivo (usa raw string o slashes normales)
ruta1 = r"C:/Users/ABY/Documents/dataset/RAECMBD_454_20230806-124929.txt"
# Ruta al archivo (usa raw string o slashes normales)
ruta2 = r"C:/Users/ABY/Documents/dataset/CMBD_454_20230806-124833.txt"

df1 = pd.read_csv(ruta1, sep=';', encoding='utf-8', low_memory=False, nrows=1000)
df2 = pd.read_csv(ruta2, sep=';', encoding='utf-8', low_memory=False, nrows=1000)

print(df1.head())
print(df2.head())

   Número de registro anual   Año    Centro Recodificado  Comunidad Autónoma  \
0                  10300924  2016   -6668509641064097861                   9   
1                  10039711  2016  -11664989471801723210                   9   
2                  10693205  2016     409303361980184901                   9   
3                  10892336  2016    4104031371584164673                   9   
4                  10161332  2016  -11664989471801723210                   9   

    Historia Recodificada  CIP Recodificado  CIP SNS Recodificado  \
0   -892136637-1183422525               NaN                   NaN   
1    -1743974610721671953               NaN                   NaN   
2  -2067360985-1925142014               NaN                   NaN   
3   -10187189121903898494               NaN                   NaN   
4     -690663659909678389               NaN                   NaN   

   Fecha de nacimiento  Edad  Sexo  ... Procedimiento 18 Procedimiento 19  \
0             28061992    2

In [11]:
# 1. Definir los grupos
grupos = {
    'arabe': ['004', '012', '031', '048', '275','364', '368', '398', '400', '414', '417', '422','430', '434', '504','512','586', '634', '682', '732', '760', '762', '784', '788', '792', '795', '818', '860', '887'],
    'caucasica': ['008','020','032','036','040','051','056','070','100','112','124','152','191','196','203','208','233','234','246','248','250','268','276','292','300','304','336','348','352','372','376','380','428',
                  '438','440','442','492','498','499','528','554','574','578','616','620','642','643','688','703','705','724','744','752','756','804','807','826','833','840'],
    'indeterminada': ['016'],
    'negra': ['024','028','044','052','060','072','076','090','092','108','120','132','136','140','148','174','175','178','180','204','212','214','226','231','232','258','262','266',
              '270','288','308','324','332','384','388','404','426','450','454','466','474','478','508','516','562','566','598','624','630','646','686','690','694','706','716','728','729','748','768','780','796','800','834','854','894'],
    'polinesia_malayo': ['050','096','104','144','184','242','254','316','360','418','446','458','520','570','583','585','608','626','652','704','764','772','776','798','876','882'],
    'china': ['064','116','156','344','392','408','410','496','524','702'],
    'hindu': ['356'],
    'hispana': ['068','084','170','188','218','222','320','340','484','558','591','600','604','862'],
    'mixta': ['192','238','296','312','328','462','470','480','500','531','533','534','535','540','548','580','581','584','612','638','654','659','660','662','663','666','670','674','678','710','740','831','832','850','858']
}

# 2. Crear un diccionario de país -> grupo
grupo_por_codigo = {}
for grupo, codigos in grupos.items():
    for codigo in codigos:
        grupo_por_codigo[codigo] = grupo

# 3. Formatear 'País Nacimiento' y asignar grupo
df1['País Nacimiento'] = df1['País Nacimiento'].astype(str).str.zfill(3)
df1['grupo_origen'] = df1['País Nacimiento'].map(grupo_por_codigo)


In [12]:
# 1 Definir diagnosticos
# Lista de columnas de diagnóstico secundario
columnas_diagnostico = [
    'Diagnóstico 2', 'Diagnóstico 3', 'Diagnóstico 4', 'Diagnóstico 5',
    'Diagnóstico 6', 'Diagnóstico 7', 'Diagnóstico 8', 'Diagnóstico 9',
    'Diagnóstico 10', 'Diagnóstico 11', 'Diagnóstico 12', 'Diagnóstico 13',
    'Diagnóstico 14', 'Diagnóstico 15', 'Diagnóstico 16', 'Diagnóstico 17',
    'Diagnóstico 18', 'Diagnóstico 19', 'Diagnóstico 20'
]

condiciones = {
    'Embarazo de alto riesgo': ['O09'],
    'Esterilidad previa': ['O09.0'],
    'Historia obstétrica adversa': ['O09.2', 'O26.2'],
    'Pérdida previa': ['O09.29', 'O26.2'],
    'Aborto previo': ['O26.21'],
    'Muerte fetal previa': ['O09.29', 'O26.23'],
    'Multípara': ['O09.4', 'O09.52', 'O09.62'],
    'Primípara': ['O09.51', 'O09.61'],
    'Embarazo TRA': ['O09.81'],
    'Embarazo TRA previo': ['O09.82'],
    'Enfermedad cardiaca hipertensiva': ['O10.1', 'O10.3'],
    'Enfermedad renal crónica hipertensiva': ['O10.2', 'O10.3'],
    'Enfermedad cardiaca y renal crónica hipertensiva': ['O10.3'],
    'Embarazo múltiple': ['O30', 'O31', 'Z37.2', 'Z37.3'],
    'HTA': ['I10', 'I11', 'I12', 'I13', 'I15', 'I16', 'I67.4', 'O10', 'O11'],
    'DM': ['E08', 'E09', 'E10', 'E11', 'O24.0', 'O24.1', 'O24.4', 'O24.8'],
    'Sobrepeso y obesidad': ['E66', 'O99.21'],
    'Tabaco': ['Z72.0', 'F17', 'T65.22', 'O99.33'],
    'Alcohol': ['F10', 'O99.31'],
    'Dislipemia': ['E78.0', 'E78.2', 'E78.4', 'E78.5'],
    'Neumopatía intersticial (genérica)': ['J84', 'J82.81', 'J82.82', '515', '516', '517.1', '517.2', '517.8'],
    'EAS intersticial (genérica)': ['J84.17', '517.1', '517.2', '517.8'],
    'Hipertensión pulmonar (genérica)': ['I27.0', 'I27.2', '416.0', '416.8'],
    'ERC (coincide con Charlson)': ['I12', 'I13', 'N18'],
    'IC (coincide con Charlson)': ['I25.5', '142', 'I50', 'I05', 'I06', 'I07', 'I08', 'I09', '134', '135', 'I36', 'I37', 'I27', 'I11', 'I13'],
    'LES': ['M32'],
    'Nefritis lúpica': ['M32.14', 'M32.15'],
    'LES-pulmón': ['M32.13'],
    'SAF v1': ['D68.61'],
    'SAF v2': ['D68.62'],
    'SAF v3': ['D68.61', 'D68.62'],
    'Portador AAF v1': ['D68.312'],
    'Portador AAF v2': ['D68.61', 'D68.312', 'D68.62'],
    'Esclerosis sistémica': ['M34'],
    'SSc-respiratorio': ['M34.81'],
    'Síndrome seco': ['M35.0'],
    'SjS-respiratorio': ['M35.02'],
    'SjS-tubulointersticial': ['M35.04'],
    'EMTC': ['M35.1'],
    'Enfermedad de Behçet': ['M35.2'],
    'Miopatía inflamatoria': ['M33', 'M36.0', 'G72.49'],
    'Vasculitis sistémica': ['M30.0', 'M30.1', 'M30.2', 'M30.3', 'M30.8', 'M31.3', 'M31.4', 'M31.5', 'M31.6', 'M31.8', 'M31.9', 'D69.0', 'D89.1'],
    'Vasculitis ANCA': ['M30.1', 'M31.3', 'M31.7'],
    'Sarcoidosis': ['D86'],
    'Artritis reumatoide': ['M05', 'M06'],
    'Artropatías enteropáticas': ['M07'],
    'Artropatia psoriásica': ['L40.5'],
    'EII': ['K50', 'K51'],
    'Enfermedad glomerular': ['N00', 'N01', 'N02', 'N03', 'N04', 'N05', 'N06', 'N08'],
    'Síndrome de Sjögren primario': ['M35.04'],
    'Antiagregación': ['Z79.02', 'Z79.82'],
    'Aspirina': ['Z79.82'],
    'Anticoagulación': ['Z79.01'],
    'Esteroides': ['Z79.52'],
    'COVID-19': ['U07.1'],
    'Neumonía COVID': ['J12.82']
}

for condicion, codigos in condiciones.items():
    patron = r'^(' + '|'.join([re.escape(cod.strip()) for cod in codigos]) + r')'
    df1[condicion] = df1[columnas_diagnostico]\
        .astype(str)\
        .apply(lambda row: any(re.match(patron, d.strip()) for d in row), axis=1)\
        .astype(int)

# Ver los primeros resultados
print(df1.head(10))


   Número de registro anual   Año    Centro Recodificado  Comunidad Autónoma  \
0                  10300924  2016   -6668509641064097861                   9   
1                  10039711  2016  -11664989471801723210                   9   
2                  10693205  2016     409303361980184901                   9   
3                  10892336  2016    4104031371584164673                   9   
4                  10161332  2016  -11664989471801723210                   9   
5                  10551498  2016  -11664989471801723210                   9   
6                   9929751  2016  -11664989471801723210                   9   
7                  10364424  2016  -11664989471801723210                   9   
8                  10560811  2016     409303361980184901                   9   
9                  10561554  2016   -1740064394191672644                   9   

    Historia Recodificada  CIP Recodificado  CIP SNS Recodificado  \
0   -892136637-1183422525               NaN       

In [13]:
#crear un fichero final

columnas = ['Comunidad Autónoma', 'Edad', 'Sexo', 'País Nacimiento', 'grupo_origen', 'Tipo Alta',
            'Ingreso en UCI', 'Días UCI', 'Estancia Días','Embarazo de alto riesgo', 
            'Esterilidad previa','Historia obstétrica adversa','Pérdida previa','Aborto previo',
            'Muerte fetal previa', 'Multípara', 'Primípara', 'Embarazo TRA','Embarazo TRA previo',
            'Enfermedad cardiaca hipertensiva','Enfermedad renal crónica hipertensiva',
            'Enfermedad cardiaca y renal crónica hipertensiva','Embarazo múltiple','HTA','DM', 
            'Sobrepeso y obesidad','Tabaco', 'Alcohol','Dislipemia', 'Neumopatía intersticial (genérica)', 
            'EAS intersticial (genérica)', 'Hipertensión pulmonar (genérica)', 'ERC (coincide con Charlson)','IC (coincide con Charlson)',
            'LES', 'Nefritis lúpica', 'LES-pulmón', 'SAF v1','SAF v2','SAF v3','Portador AAF v1','Portador AAF v2','Esclerosis sistémica',
            'SSc-respiratorio','Síndrome seco', 'SjS-respiratorio','SjS-tubulointersticial','EMTC', 'Enfermedad de Behçet', 'Miopatía inflamatoria',
            'Vasculitis sistémica','Vasculitis ANCA', 'Sarcoidosis','Artritis reumatoide', 'Artropatías enteropáticas','Artropatia psoriásica','EII', 
            'Enfermedad glomerular','Síndrome de Sjögren primario','Antiagregación','Aspirina','Anticoagulación','Esteroides','COVID-19','Neumonía COVID']
df_final = df1[columnas].copy()
# 4. Ver resultado
print(df_final.head())


   Comunidad Autónoma  Edad  Sexo País Nacimiento grupo_origen  Tipo Alta  \
0                   9    24     2             604      hispana          1   
1                   9    31     2             250    caucasica          1   
2                   9    35     2             352    caucasica          1   
3                   9    31     2             724    caucasica          3   
4                   9    32     2             250    caucasica          1   

   Ingreso en UCI  Días UCI  Estancia Días  Embarazo de alto riesgo  ...  \
0               2       NaN              4                        1  ...   
1               2       NaN              3                        0  ...   
2               2       NaN              4                        0  ...   
3               2       NaN              2                        1  ...   
4               2       NaN              3                        0  ...   

   Artropatia psoriásica  EII  Enfermedad glomerular  \
0                      0

In [14]:
# Guardar el DataFrame en un archivo CSV
df_final.to_csv("fichero_final.csv", index=False, encoding='utf-8-sig')